# Monoidal Trees

Based on Brent A. Yorgey. 2012. Monoids: theme and variations (functional pearl). SIGPLAN Not. 47, 12 (December 2012), 105–116. https://doi.org/10.1145/2430532.2364520

Also see https://github.com/diagrams/dual-tree


In [12]:
import $ivy.`org.typelevel::cats-core:2.1.0`

import $ivy.$                               

In [13]:
import cats._
import cats.implicits._

trait Action[M, A] extends Monoid[M] {
  def act(a: A): A
}

sealed trait MTree[D, U, L] {
  def u: U
} 

case class Leaf[U, L](u: U, l: L) extends MTree[Nothing, U, L]

case class Concat[D, U, L](u: U, ts: List[MTree[D, U, L]]) extends MTree[D, U, L]

case class Act[D, U, L](d: D, t: MTree[D, U, L])(implicit act: (D, U) => U) extends MTree[D, U, L] {
  def u: U = act(d, t.u)
}

import cats._

import cats.implicits._


defined trait Action
defined trait MTree
defined class Leaf
defined class Concat
defined class Act

In [14]:
trait Prim

case class Circle(r: Double) extends Prim
case class Rectangle(w: Double, h: Double) extends Prim

type Diagram = List[Prim]

val demo1 = List(Circle(1), Rectangle(2, 3))
val demo2 = List(Circle(4))
demo1 |+| demo2

defined trait Prim
defined class Circle
defined class Rectangle
defined type Diagram
demo1: List[Product with Serializable with Prim] = List(
  Circle(1.0),
  Rectangle(2.0, 3.0)
)
demo2: List[Circle] = List(Circle(4.0))
res13_6: List[Product with Serializable with Prim] = List(
  Circle(1.0),
  Rectangle(2.0, 3.0),
  Circle(4.0)
)

In [58]:
case class V2(x: Double, y: Double) {
  lazy val len: Double = math.sqrt(x * x + y * y)
  
  def *(m: Double) = V2(x * m, y * m)
  
  def unary_- : V2 = V2(-x, -y)
  
  def dot(v: V2): Double = x * v.x + y * v.y
}

case class P2(x: Double, y: Double) {
  def +(v: V2): P2 = P2(x + v.x, y + v.y)
  
  def -(p: P2): V2 = V2(x - p.x, y - p.y)
}

type Envelope = V2 => Double

type Trace = P2 => V2 => Option[Double]

trait Segment {
  def translate(v: V2): Segment
  
  def envelope: Envelope
  
  def trace: Trace
}

case class LineSegment(p0: P2, p1: P2) extends Segment {
  def translate(v: V2): Segment = LineSegment(p0 + v, p1 + v)
  
  def envelope: Envelope = { v =>
    val d0 = p0.x * v.x + p0.y * v.y
    val d1 = p1.x * v.x + p1.y * v.y
    (d0 max d1) / (v dot v)
  }
  
  lazy val trace: Trace = p => v => {
    // see https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection#Given_two_points_on_each_line
    // L1: p + v*t
    // L2: p0 + (p1 - p0)*u
    val w = p1 - p0
    val det = v.x * w.y - v.y * w.x
    if (det == 0) {
      // L1 and L2 are parallel
      if (v.x * (p.x - p0.x) == v.y * (p.y - p0.y)) {
        // L1 and L2 coincide; find which of p0 and p1 is closer to p
        val v0 = p0 - p
        val v1 = p1 - p
        if (v0.len <= v1.len) {
          Some(v0.len / v.len)
        } else {
          Some(v1.len / v.len)
        }
      } else {
        // No intersection
        None
      }
    } else {
      // Find intersection point along L2
      val u = (v.x * (p.y - p0.y) - v.y * (p.x - p0.x)) / det
      if (u < 0 || u > 1) {
        // Intersection outside the segment
        None
      } else {
        // Find distance along L1 to intersection
        val t = ((p.x - p0.x) * (p0.y - p1.y) - (p.y - p0.y) * (p0.x - p1.x)) / det
        Some(t)
      }
    }
  }
}

// TODO ArcSegment, QBezSegment, CBezSegment

trait Diagram {
  def translate(v: V2): Diagram
  
  def envelope: Option[Envelope]
  
  def trace: Trace
  
  def over(d: Diagram): Diagram = Over(this, d)
  
  def beside(v: V2, d: Diagram): Diagram = (envelope, d.envelope) match {
    case (Some(te), Some(de)) => {
      val vtrans = v * (te(v) + de(-v))
      over(d.translate(vtrans))
    }
    case _ => over(d)
  }
}

case object Empty extends Diagram {
  def translate(v: V2): Diagram = this
  
  val envelope: Option[Envelope] = None
  
  val trace: Trace = p => v => None
  
  override def over(d: Diagram): Diagram = d
}

case class Path(segments: List[Segment]) extends Diagram {
  def translate(v: V2): Diagram = Path(segments.map(_.translate(v)))
  
  lazy val envelope: Option[Envelope] = Some(v => segments.map(_.envelope(v)).max)
  
  lazy val trace: Trace = p => v => {
    val d = segments.map(_.trace(p)(v).getOrElse(Double.PositiveInfinity)).min
    if (d.isInfinity) {
      None
    } else {
      Some(d)
    }
  }
}

case class Over(a: Diagram, b: Diagram) extends Diagram {
  def translate(v: V2): Diagram = Over(a.translate(v), b.translate(v))
  
  lazy val envelope: Option[Envelope] = (a.envelope, b.envelope) match {
    case (Some(ae), Some(be)) => Some(v => ae(v) max be(v))
    case (e, None) => e
    case (None, e) => e
    case _ => None
  }
  
  lazy val trace: Trace = p => v => (a.trace(p)(v), b.trace(p)(v)) match {
    case (Some(ad), Some(bd)) => Some(ad min bd)
    case (x, None) => x
    case (None, x) => x
    case _ => None
  }
}

object Diagram {
  implicit val diagramMonoid: Monoid[Diagram] =
    new Monoid[Diagram] {
      def empty: Diagram = Empty
      
      def combine(a: Diagram, b: Diagram): Diagram = a.over(b)
    }
}

defined class V2
defined class P2
defined type Envelope
defined type Trace
defined trait Segment
defined class LineSegment
defined trait Diagram
defined object Empty
defined class Path
defined class Over
defined object Diagram

In [67]:
def rectangle(w: Double, h: Double): Diagram = {
  val ul = P2(-w/2, h/2)
  val ur = P2(w/2, h/2)
  val ll = P2(-w/2, -h/2)
  val lr = P2(w/2, -h/2)
  Path(List(
    LineSegment(ul, ur),
    LineSegment(ur, lr),
    LineSegment(lr, ll),
    LineSegment(ll, ul)
  ))
}

val r1 = rectangle(2, 3)
val Some(e) = r1.envelope
e(V2(2, 0))
e(V2(2, 1))
e(V2(2, 2))
e(V2(2, 3))
e(V2(1, 3))
e(V2(0, 3))
e(V2(-1, 3))
e(V2(2, -1))
e(V2(2, -2))
e(V2(2, -3))
e(V2(1, -3))
e(V2(0, -3))
r1.trace(P2(-2, 0))(V2(0.5, 0.74))
r1.trace(P2(-2, 0))(V2(0.5, 0.75))
r1.trace(P2(-2, 0))(V2(0.5, 0.76))

defined function rectangle
r1: Diagram = Path(
  List(
    LineSegment(P2(-1.0, 1.5), P2(1.0, 1.5)),
    LineSegment(P2(1.0, 1.5), P2(1.0, -1.5)),
    LineSegment(P2(1.0, -1.5), P2(-1.0, -1.5)),
    LineSegment(P2(-1.0, -1.5), P2(-1.0, 1.5))
  )
)
e: V2 => Double = ammonite.$sess.cmd57$Helper$Path$$Lambda$5174/0x0000000801d5f840@37259cd
res66_3: Double = 0.5
res66_4: Double = 0.7
res66_5: Double = 0.625
res66_6: Double = 0.5
res66_7: Double = 0.55
res66_8: Double = 0.5
res66_9: Double = 0.55
res66_10: Double = 0.7
res66_11: Double = 0.625
res66_12: Double = 0.5
res66_13: Double = 0.55
res66_14: Double = 0.5
res66_15: Option[Double] = Some(2.0)
res66_16: Option[Double] = Some(2.0)
res66_17: Option[Double] = None

In [68]:
val r2 = rectangle(1, 4)
val d2 = r1 |+| r2
val Some(e2) = d2.envelope
e2(V2(2, 0))
e2(V2(2, 1))
e2(V2(2, 2))
e2(V2(2, 3))
e2(V2(1, 3))
e2(V2(0, 3))
e2(V2(-1, 3))
e2(V2(2, -1))
e2(V2(2, -2))
e2(V2(2, -3))
e2(V2(1, -3))
e2(V2(0, -3))

r2: Diagram = Path(
  List(
    LineSegment(P2(-0.5, 2.0), P2(0.5, 2.0)),
    LineSegment(P2(0.5, 2.0), P2(0.5, -2.0)),
    LineSegment(P2(0.5, -2.0), P2(-0.5, -2.0)),
    LineSegment(P2(-0.5, -2.0), P2(-0.5, 2.0))
  )
)
d2: Diagram = Over(
  Path(
    List(
      LineSegment(P2(-1.0, 1.5), P2(1.0, 1.5)),
      LineSegment(P2(1.0, 1.5), P2(1.0, -1.5)),
      LineSegment(P2(1.0, -1.5), P2(-1.0, -1.5)),
      LineSegment(P2(-1.0, -1.5), P2(-1.0, 1.5))
    )
  ),
  Path(
    List(
      LineSegment(P2(-0.5, 2.0), P2(0.5, 2.0)),
      LineSegment(P2(0.5, 2.0), P2(0.5, -2.0)),
      LineSegment(P2(0.5, -2.0), P2(-0.5, -2.0)),
      LineSegment(P2(-0.5, -2.0), P2(-0.5, 2.0))
    )
  )
)
e2: V2 => Double = ammonite.$sess.cmd57$Helper$Over$$Lambda$5477/0x0000000801e02040@28c0991f
res67_3: Double = 0.5
res67_4: Double = 0.7
res67_5: Double = 0.625
res67_6: Double = 0.5384615384615384
res67_7: Double = 0.65
res67_8: Double = 0.6666666666666666
res67_9: Double = 0.65
res67_10: Double = 0.7
res67_

In [69]:
val d3 = r1.beside(V2(1, 0), r2)
val Some(e3) = d3.envelope
e3(V2(2, 0))
e3(V2(2, 1))
e3(V2(2, 2))
e3(V2(2, 3))
e3(V2(1, 3))
e3(V2(0, 3))
e3(V2(-1, 3))
e3(V2(2, -1))
e3(V2(2, -2))
e3(V2(2, -3))
e3(V2(1, -3))
e3(V2(0, -3))

d3: Diagram = Over(
  Path(
    List(
      LineSegment(P2(-1.0, 1.5), P2(1.0, 1.5)),
      LineSegment(P2(1.0, 1.5), P2(1.0, -1.5)),
      LineSegment(P2(1.0, -1.5), P2(-1.0, -1.5)),
      LineSegment(P2(-1.0, -1.5), P2(-1.0, 1.5))
    )
  ),
  Path(
    List(
      LineSegment(P2(1.0, 2.0), P2(2.0, 2.0)),
      LineSegment(P2(2.0, 2.0), P2(2.0, -2.0)),
      LineSegment(P2(2.0, -2.0), P2(1.0, -2.0)),
      LineSegment(P2(1.0, -2.0), P2(1.0, 2.0))
    )
  )
)
e3: V2 => Double = ammonite.$sess.cmd57$Helper$Over$$Lambda$5477/0x0000000801e02040@1a1bdcb9
res68_2: Double = 1.0
res68_3: Double = 1.2
res68_4: Double = 1.0
res68_5: Double = 0.7692307692307693
res68_6: Double = 0.8
res68_7: Double = 0.6666666666666666
res68_8: Double = 0.55
res68_9: Double = 1.2
res68_10: Double = 1.0
res68_11: Double = 0.7692307692307693
res68_12: Double = 0.8
res68_13: Double = 0.6666666666666666